In [10]:
# Import General Relevant Libraries
import os
import time
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Prepare list of file names for efficiently loading each prediction file later on. 
groundlist = ['cc256_ga_testlabels.npy', 'cc512_ga_testlabels.npy', 'cc256_ga_testlabels.npy', 'cc512_ga_testlabels.npy', 'cc256_ga_testlabels.npy', 'cc512_ga_testlabels.npy', 'jde256_ga_testlabels.npy', 'jde512_ga_testlabels.npy', 'jde256_ga_testlabels.npy', 'jde512_ga_testlabels.npy', 'jde256_ga_testlabels.npy', 'jde512_ga_testlabels.npy', 'jdi256_ga_testlabels.npy', 'jdi512_ga_testlabels.npy', 'jdi256_ga_testlabels.npy', 'jdi512_ga_testlabels.npy', 'jdi256_ga_testlabels.npy', 'jdi512_ga_testlabels.npy', 'sa256_ga_testlabels.npy', 'sa512_ga_testlabels.npy', 'sa256_ga_testlabels.npy', 'sa512_ga_testlabels.npy', 'sa256_ga_testlabels.npy', 'sa512_ga_testlabels.npy']
featurelist = ['cc256_ga_test.npy', 'cc512_ga_test.npy', 'cc256_ga_test.npy', 'cc512_ga_test.npy', 'cc256_ga_test.npy', 'cc512_ga_test.npy', 'jde256_ga_test.npy', 'jde512_ga_test.npy', 'jde256_ga_test.npy', 'jde512_ga_test.npy', 'jde256_ga_test.npy', 'jde512_ga_test.npy', 'jdi256_ga_test.npy', 'jdi512_ga_test.npy', 'jdi256_ga_test.npy', 'jdi512_ga_test.npy', 'jdi256_ga_test.npy', 'jdi512_ga_test.npy', 'sa256_ga_test.npy', 'sa512_ga_test.npy', 'sa256_ga_test.npy', 'sa512_ga_test.npy', 'sa256_ga_test.npy', 'sa512_ga_test.npy']
predlist = ['pred_stack_og_cc256g.npy', 'pred_stack_og_cc512g.npy', 'pred_stack_p2_cc256g.npy', 'pred_stack_p2_cc512g.npy', 'pred_stack_r_cc256g.npy', 'pred_stack_r_cc512g.npy', 'pred_stack_og_jde256g.npy', 'pred_stack_og_jde512g.npy', 'pred_stack_p2_jde256g.npy', 'pred_stack_p2_jde512g.npy', 'pred_stack_r_jde256g.npy', 'pred_stack_r_jde512g.npy', 'pred_stack_og_jdi256g.npy', 'pred_stack_og_jdi512g.npy', 'pred_stack_p2_jdi256g.npy', 'pred_stack_p2_jdi512g.npy', 'pred_stack_r_jdi256g.npy', 'pred_stack_r_jdi512g.npy', 'pred_stack_og_sa256g.npy', 'pred_stack_og_sa512g.npy', 'pred_stack_p2_sa256g.npy', 'pred_stack_p2_sa512g.npy', 'pred_stack_r_sa256g.npy', 'pred_stack_r_sa512g.npy']

In [12]:
# Sanity Check: check matching ground files and predicted files, in the preferred order. 
print(len(groundlist),len(featurelist),len(predlist))
print()
for a, b in zip(groundlist, predlist):
    print(a, b)

24 24 24

cc256_ga_testlabels.npy pred_stack_og_cc256g.npy
cc512_ga_testlabels.npy pred_stack_og_cc512g.npy
cc256_ga_testlabels.npy pred_stack_p2_cc256g.npy
cc512_ga_testlabels.npy pred_stack_p2_cc512g.npy
cc256_ga_testlabels.npy pred_stack_r_cc256g.npy
cc512_ga_testlabels.npy pred_stack_r_cc512g.npy
jde256_ga_testlabels.npy pred_stack_og_jde256g.npy
jde512_ga_testlabels.npy pred_stack_og_jde512g.npy
jde256_ga_testlabels.npy pred_stack_p2_jde256g.npy
jde512_ga_testlabels.npy pred_stack_p2_jde512g.npy
jde256_ga_testlabels.npy pred_stack_r_jde256g.npy
jde512_ga_testlabels.npy pred_stack_r_jde512g.npy
jdi256_ga_testlabels.npy pred_stack_og_jdi256g.npy
jdi512_ga_testlabels.npy pred_stack_og_jdi512g.npy
jdi256_ga_testlabels.npy pred_stack_p2_jdi256g.npy
jdi512_ga_testlabels.npy pred_stack_p2_jdi512g.npy
jdi256_ga_testlabels.npy pred_stack_r_jdi256g.npy
jdi512_ga_testlabels.npy pred_stack_r_jdi512g.npy
sa256_ga_testlabels.npy pred_stack_og_sa256g.npy
sa512_ga_testlabels.npy pred_stack_og_sa5

In [8]:
# Information about predictions files: 
"""
Prediction files consist of arrays with a identical shape as the testlabel arrays. 
Wlements are numbers between 0 and 1 that are output from the sigmoid activation function. 
We consider values (representing pixels) equal or greater than 0.5 part of the label. 
"""

''

In [13]:
# Define function for converting the prediction values to binary values (0=not part of label, 1= part of label)
def threshold_to_binary(array, threshold=0.5):
    return (array >= threshold)

In [8]:
### Computing the IoU score of each prediction file

array = []
for a, b in zip(groundlist, predlist):
    # Specify Input Path of ground truth
    os.chdir('C:/Users/quint/OneDrive/qthesis/02_datasets/matrices/train_test_matrices/greedy/')
    ground = np.load(a)
    # Specify Input Path of prediction
    os.chdir('C:/Users/quint/OneDrive/qthesis/02_datasets/matrices/prediction_matrices/')
    pred = np.load(b)

    pred = threshold_to_binary(pred) # Convert prediction values to binary values
    
    # Compute the number of TPs, TNs, FPs, and FNs per label. 
    TP = np.sum(np.logical_and(pred == 1, ground == 1), axis=(0, 1, 2))
    TN = np.sum(np.logical_and(pred == 0, ground == 0), axis=(0, 1, 2))
    FP = np.sum(np.logical_and(pred == 1, ground == 0), axis=(0, 1, 2))
    FN = np.sum(np.logical_and(pred == 0, ground == 1), axis=(0, 1, 2))
    # Each variable has a shape of (5,)
    #print(TP.shape, TN.shape, FP.shape, FN.shape)

    # Compute the IoU score for each label
    IoU_labels = []
    for i in range(5):
        IoU = TP[i] / (TP[i] + FP[i] + FN[i])
        IoU_labels.append(IoU)
    print(f'{IoU_labels},')
    array.append(IoU_labels)

iou_array = np.array(array)

[0.7138140270119578, 0.9447440636081135, 0.9584012902502027, 0.8781650719144819, 0.8555313535018441],
[0.44707649625173396, 0.8557559260887408, 0.9286614855239944, 0.8189874185795579, 0.6330052537075215],
[0.5837402804832643, 0.9467238809777536, 0.9606070346383595, 0.8700063109572554, 0.8587045563941186],
[0.21647123126671522, 0.87530490092225, 0.9383666919566385, 0.8207068003148493, 0.4124513427822511],
[0.7552206914387439, 0.9483009742323425, 0.9600182657026745, 0.8810945913902439, 0.8638881729824178],
[0.6319368548876746, 0.9241523959448436, 0.9526608646930542, 0.8602828437082752, 0.774800570215541],
[0.7705856211389303, 0.9453677340813155, 0.7928980048399574, 0.8763211652942063, 0.7909491531994908],
[0.598721405162811, 0.9178414970723895, 0.7844983604946763, 0.8317128894625888, 0.7203484542932413],
[0.7414670613788937, 0.9440961456395656, 0.7264422854921484, 0.8743067835410747, 0.7693589111203833],
[0.41608606396815695, 0.9060776718441002, 0.7012922120482579, 0.8244124259747448, 0.

In [10]:
# Convert IoU decimal-numbers to percentages, rounded to 2 decimals. 
iou_array = np.around((iou_array*100),2)
print(iou_array)

[[71.38 94.47 95.84 87.82 85.55]
 [44.71 85.58 92.87 81.9  63.3 ]
 [58.37 94.67 96.06 87.   85.87]
 [21.65 87.53 93.84 82.07 41.25]
 [75.52 94.83 96.   88.11 86.39]
 [63.19 92.42 95.27 86.03 77.48]
 [77.06 94.54 79.29 87.63 79.09]
 [59.87 91.78 78.45 83.17 72.03]
 [74.15 94.41 72.64 87.43 76.94]
 [41.61 90.61 70.13 82.44 44.19]
 [73.32 94.77 77.6  87.67 79.23]
 [56.06 91.52 77.01 84.14 72.14]
 [35.75 85.96 96.13 82.13 88.43]
 [ 3.33 79.82 96.05 79.   85.57]
 [18.15 77.94 95.2  76.77 78.6 ]
 [ 0.   63.82 95.29 65.89 47.33]
 [38.16 86.41 96.24 79.31 87.68]
 [ 4.21 79.21 95.76 77.51 80.04]
 [75.36 93.39 95.16 87.02 85.79]
 [57.94 90.97 94.61 84.53 76.54]
 [70.06 93.59 95.33 87.26 83.82]
 [41.74 89.45 93.99 83.09 69.94]
 [70.41 93.52 95.   87.19 83.18]
 [51.38 88.88 94.04 82.58 74.03]]


In [17]:
### Computing the F-beta score of each prediction file

beta = 0.707
array = []
for a, b in zip(groundlist, predlist):
    # Specify Input Path of ground truth
    os.chdir('C:/Users/quint/OneDrive/qthesis/02_datasets/matrices/train_test_matrices/greedy/')
    ground = np.load(a)
    # Specify Input Path of prediction
    os.chdir('C:/Users/quint/OneDrive/qthesis/02_datasets/matrices/prediction_matrices/')
    pred = np.load(b)

    pred = threshold_to_binary(pred) # Convert prediction values to binary values

    # Compute the number of TPs, TNs, FPs, and FNs per label. 
    TP = np.sum(np.logical_and(pred == 1, ground == 1), axis=(0, 1, 2))
    TN = np.sum(np.logical_and(pred == 0, ground == 0), axis=(0, 1, 2))
    FP = np.sum(np.logical_and(pred == 1, ground == 0), axis=(0, 1, 2))
    FN = np.sum(np.logical_and(pred == 0, ground == 1), axis=(0, 1, 2))
    # Each variable has a shape of (5,)
    #print(TP.shape, TN.shape, FP.shape, FN.shape)

    # Compute the F-beta score for each label
    FB_labels = []
    for i in range(5):
        precision = TP[i] / (TP[i] + FP[i]) if TP[i] + FP[i] > 0 else 0
        recall = TP[i] / (TP[i] + FN[i]) if TP[i] + FN[i] > 0 else 0
        FB = (1 + beta**2) * precision * recall / (beta**2 * precision + recall) if (beta**2 * precision + recall) > 0 else 0
        FB_labels.append(FB)
    print(f'{FB_labels},')
    array.append(FB_labels)

fb_array = np.array(array)

[0.8330173324969875, 0.970310849591037, 0.9777998851708954, 0.9345037237420585, 0.9263995448537338],
[0.6574472571063706, 0.9408391979708092, 0.9706257291465907, 0.9085344219488927, 0.7855604368285571],
[0.7918258083103924, 0.9741208449461902, 0.9808339436302953, 0.9270784120162753, 0.9321381769878226],
[0.4346005466198815, 0.9425240893449327, 0.9695809624230837, 0.9024370859803803, 0.6517540720511599],
[0.8644457930445041, 0.9747986416891816, 0.9808888423410025, 0.9348406665526791, 0.9221960537377517],
[0.7817902033932245, 0.9628958399449331, 0.9757429136558452, 0.9266737537441669, 0.8730909354483958],
[0.8792030867658682, 0.9734514030341662, 0.8887709490005112, 0.9394098996400587, 0.8917674579067383],
[0.7844583216403224, 0.9526488629451737, 0.8744080896046983, 0.9175797828756865, 0.8346821862594728],
[0.8428533558920399, 0.9701746782501584, 0.8424187789762673, 0.9337379764346001, 0.8756465142447497],
[0.6398074388202001, 0.9471782127094126, 0.8365020581670954, 0.9096483592975633, 0.

In [11]:
# Convert Fbeta decimal-numbers to percentages, rounded to 2 decimals. 
fb_array = np.around((fb_array*100),2)
print(fb_array)

[[83.3  97.03 97.78 93.45 92.64]
 [65.74 94.08 97.06 90.85 78.56]
 [79.18 97.41 98.08 92.71 93.21]
 [43.46 94.25 96.96 90.24 65.18]
 [86.44 97.48 98.09 93.48 92.22]
 [78.18 96.29 97.57 92.67 87.31]
 [87.92 97.35 88.88 93.94 89.18]
 [78.45 95.26 87.44 91.76 83.47]
 [84.29 97.02 84.24 93.37 87.56]
 [63.98 94.72 83.65 90.96 68.23]
 [87.16 97.49 87.17 93.87 89.14]
 [77.81 95.35 86.25 90.72 82.58]
 [58.08 93.61 97.97 90.17 93.59]
 [ 9.31 90.52 97.81 88.25 91.76]
 [37.3  88.4  97.78 87.86 87.05]
 [ 0.   80.24 97.74 83.56 67.77]
 [60.89 92.8  98.21 90.38 93.26]
 [11.55 88.79 97.83 88.3  89.43]
 [86.24 96.4  97.45 92.83 92.6 ]
 [75.03 95.28 97.18 91.56 87.34]
 [83.74 96.82 97.73 93.43 91.78]
 [64.41 94.75 96.91 90.54 83.63]
 [83.97 96.68 97.32 93.54 91.55]
 [69.39 94.23 97.19 90.93 85.56]]


In [17]:
# Find the highest scores for each evaluation metric.

iou_max = np.amax(iou_array, axis=0)
print(iou_max)

fb_max = np.amax(fb_array, axis=0)
print(fb_max)

[77.06 94.83 96.24 88.11 88.43]
[87.92 97.49 98.21 93.94 93.59]
